In [1]:
%pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 12.1 MB/s eta 0:00:00


In [5]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from tqdm import tqdm  # Import tqdm for the progress bar
import google_colab_selenium as gs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Base URL and headers remain the same
base_url = 'https://www.ultimatewindowssecurity.com/securitylog/encyclopedia/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

def scrape_event_details(event_url):
    # Event details scraping function remains the same
    response = requests.get(event_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    os_label = soup.find('span', id='ctl00_ctl00_ctl00_ctl00_Content_Content_Content_Content_lblOSorSourceLabel')
    if os_label:
        os_text = os_label.find_next('td').text.strip() if os_label.find_next('td') else 'Not available'
        os_data = re.split(r'\s+and\s+', os_text)
    else:
        os_data = ['Not available']

    category_span = soup.find('span', id='ctl00_ctl00_ctl00_ctl00_Content_Content_Content_Content_categories')
    category_data = category_span.text.strip() if category_span else 'Not available'

    type_row = soup.find('tr', id='ctl00_ctl00_ctl00_ctl00_Content_Content_Content_Content_trType')
    type_data = type_row.find_all('td')[1].text.strip() if type_row and len(type_row.find_all('td')) > 1 else 'Not available'

    description = soup.find('ul').find_next_sibling('p')
    description = description.text.strip() if description else 'No description available'

    event_example = soup.find('p', class_='EventExample')
    event_example = event_example.text.strip() if event_example else 'No example available'

    return os_data, category_data, type_data, description, event_example

def handle_cookie_consent(driver):
    try:
        # Try to remove the cookie consent div using JavaScript
        driver.execute_script("""
            var cookieConsent = document.getElementById('ctl00_ctl00_ctl00_ctl00_CookieConsent');
            if(cookieConsent) {
                cookieConsent.parentNode.removeChild(cookieConsent);
            }
        """)
        time.sleep(1)  # Brief pause to let the DOM update
    except Exception as e:
        print(f"Error handling cookie consent: {str(e)}")

    # Also remove any potential overlay elements
    driver.execute_script("""
        var elements = document.getElementsByTagName('div');
        for(var i = 0; i < elements.length; i++) {
            if(elements[i].style.zIndex > 1000) {
                elements[i].parentNode.removeChild(elements[i]);
            }
        }
    """)

def main():
    driver = gs.Chrome()
    try:
        # Navigate to the webpage
        driver.get('https://www.ultimatewindowssecurity.com/securitylog/encyclopedia/default.aspx')

        # Handle cookie consent first
        handle_cookie_consent(driver)

        # Click the radio button using JavaScript
        driver.execute_script("""
            var radioBtn = document.getElementById('ctl00_ctl00_ctl00_ctl00_Content_Content_Content_Content_whichEvents_0');
            if(radioBtn) {
                radioBtn.click();
            }
        """)

        # Wait for the page to update
        time.sleep(2)

        # Get the updated page source
        updated_page_source = driver.page_source
        soup = BeautifulSoup(updated_page_source, 'html.parser')

        table = soup.find('table', id='ctl00_ctl00_ctl00_ctl00_Content_Content_Content_Content_GridView1')
        rows = table.find_all('tr')[1:]

        data = []
        existing_data = []

        if os.path.exists('detailed_events.csv'):
            existing_df = pd.read_csv('detailed_events.csv')
            existing_data = existing_df['Event_ID'].astype(str).tolist()

        rows_to_process = [row for row in rows if row.find_all('td')[1].text.strip() not in existing_data]

        progress_bar = tqdm(total=len(rows_to_process), desc="Scraping rows", dynamic_ncols=True)

        for row in rows_to_process:
            cols = row.find_all('td')
            if cols:
                source = cols[0].text.strip()
                event_id = cols[1].text.strip()
                event_summary = cols[2].text.strip()
                event_url = base_url + cols[2].find('a')['href']
                os_data, category_data, type_data, description, event_example = scrape_event_details(event_url)
                data.append([source, event_id, event_summary, event_url, os_data, category_data, type_data, description, event_example])
                time.sleep(random.randint(1, 3))  # Reduced sleep time
                progress_bar.update(1)

        progress_bar.close()

        df = pd.DataFrame(data, columns=['Source', 'Event_ID', 'Event Summary', 'URL', 'Operating Systems',
                                       'Category and Subcategory', 'Type', 'Description', 'Event Example'])

        if os.path.exists('detailed_events.csv'):
            existing_df = pd.read_csv('detailed_events.csv')
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df.to_csv('detailed_events.csv', index=False)
        else:
            df.to_csv('detailed_events.csv', index=False)

    finally:
        driver.quit()

if __name__ == '__main__':
    main()

<IPython.core.display.Javascript object>

Scraping rows: 100%|██████████| 573/573 [23:17<00:00,  2.44s/it]


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL to scrape
url = 'https://learn.microsoft.com/en-us/windows-server/identity/ad-ds/plan/appendix-l--events-to-monitor'

# Fetch the page
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables
tables = soup.find_all('table')

# Initialize an empty list to hold all rows from all tables
all_rows = []

# Iterate over each table
for table in tables:
    # Find all rows in the current table
    rows = table.find_all('tr')
    # Skip the header row
    rows = rows[1:]
    # Extract data from each row
    for row in rows:
        cols = row.find_all('td')
        if cols:
            current_id = cols[0].text.strip()
            legacy_id = cols[1].text.strip()
            criticality = cols[2].text.strip()
            summary = cols[3].text.strip()
            # Normalize Legacy_Event_ID
            legacy_ids = re.split(r',\s*', legacy_id)
            expanded_ids = []
            for id_range in legacy_ids:
                if '-' in id_range:
                    start, end = map(int, id_range.split('-'))
                    expanded_ids.extend(map(str, range(start, end + 1)))
                else:
                    expanded_ids.append(id_range)
            normalized_legacy_ids = ','.join(expanded_ids)
            all_rows.append([current_id, normalized_legacy_ids, criticality, summary])

# Create a DataFrame
new_df = pd.DataFrame(all_rows, columns=['Current_Event_ID', 'Legacy_Event_ID', 'Potential_Criticality', 'Event_Summary'])

In [7]:
# Load the existing CSV
existing_df = pd.read_csv('detailed_events.csv')

# Convert the 'Current_Event_ID' column in 'new_df' to int64
new_df['Current_Event_ID'] = pd.to_numeric(new_df['Current_Event_ID'], errors='coerce')

# Merge the new data based on Event_ID using left join
merged_df = existing_df.merge(new_df, left_on='Event_ID', right_on='Current_Event_ID', how='left')

# Drop the extra columns if not needed
merged_df.drop(columns=['Current_Event_ID'], inplace=True)

# Rename the 'Event_Summary_y' column to 'Event_Summary' if it exists
if 'Event_Summary_y' in merged_df.columns:
    merged_df.rename(columns={'Event_Summary_y': 'Event_Summary'}, inplace=True)

# Drop the 'Event_Summary_x' column if it exists
if 'Event_Summary_x' in merged_df.columns:
    merged_df.drop(columns=['Event_Summary_x'], inplace=True)

# Remove '000' from the 'Legacy_Event_ID' column
merged_df['Legacy_Event_ID'] = merged_df['Legacy_Event_ID'].apply(lambda x: ','.join([id for id in str(x).split(',') if id.strip() != '000']))

# Save the updated DataFrame
merged_df.to_csv('updated_detailed_events.csv', index=False)

In [8]:
import csv

# Load the existing CSV
df = pd.read_csv('updated_detailed_events.csv')

# Ensure the 'Operating Systems' column is read as a string
df['Operating Systems'] = df['Operating Systems'].astype(str)

# Remove new lines from the 'Operating Systems' column and replace values
df['Operating Systems'] = df['Operating Systems'].apply(lambda x: x.replace('\\r\\n', ' ')
                                                                   .replace('2022', 'Windows Server 2022')
                                                                   .replace('7', 'Windows 7')
                                                                   .replace('8.1', 'Windows 8.1')
                                                                   .replace('Windows 2003', 'Windows Server 2003')
                                                                   .replace('Windows 2016', 'Windows Server 2016')
                                                                   .replace('10', 'Windows 10')
                                                                   .replace('XP', 'Windows XP'))

# Split the 'Operating Systems' string using regular expressions and join with commas
df['Operating Systems'] = df['Operating Systems'].apply(lambda x: ', '.join(re.findall(r'Windows (?:Server )?(?:2008 R2|2012 R2|2000|2003|2016|2019|2022|7|8\.1|10|XP)', x)))

# Remove undesired characters from all columns
df = df.applymap(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)) if isinstance(x, str) else x)

# Remove the specific text pattern from the 'Description' column
df['Description'] = df['Description'].apply(lambda x: re.sub(r"I haven't been able to produce this event\. Have you\? If so, please start a discussion \(see above\) and post a sample along with any comments you may have! Don't forget to sanitize any private information\.", '', str(x)))

# Drop rows where all values are NaN
df.dropna(how='all', inplace=True)

# Save the updated DataFrame
df.to_csv('updated_detailed_events.csv', index=False)

<ipython-input-8-ba6426bf1c27>:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)) if isinstance(x, str) else x)
